## Questo notebook scarica il calendario dividends da investing.com
## Le date sono settabili come riportato sotto





In [ ]:
#mounts current gdrive in colab after providing success code
#INSERISCI IL CODICE DI SUCCESSO O NON FUNZIONERA'
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/My Drive/INVEST-IO/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/INVEST-IO


In [ ]:
#apply filter on dates, scarica sempre dallo startdate (da indicare sotto) a enddate
#se enddate = today alla viene scaricato fino ad oggi dinamico, altrimenti imposta enddate alla data desiderata
#!!! FORMATO DATE US !!!
startdate = '01/01/2020'
enddate = '01/31/2020'

#importa la data di oggi nel formato corretto
from datetime import date
import time
if enddate == 'today':
  today = date.today()
  today_us_format = today.strftime("%m/%d/%Y")
else:
  today_us_format = enddate

In [3]:
%%capture
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from tqdm import tqdm_notebook as progress_bar
import pandas
import json

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Calcula

WebDriverException: ignored

In [ ]:
#function per cliccare pulsanti su chrome .click() di selenium non funziona
def click_on_button_id(x):
  element = wd.find_element_by_id(x)
  wd.execute_script("arguments[0].click();", element)

In [ ]:
#CALENDARIO DIVIDENDS

url = 'https://www.investing.com/dividends-calendar/'

wd.set_window_size(1920,1080)
wd.maximize_window()
wd.get(url)

#clicca sui pulsanti per filtrare la data
click_on_button_id('datePickerToggleBtn')
click_on_button_id('startDate')
wd.find_element_by_id("startDate").clear()
wd.find_element_by_id("startDate").send_keys(startdate)
click_on_button_id('endDate')
wd.find_element_by_id("endDate").clear()
wd.find_element_by_id("endDate").send_keys(today_us_format)
click_on_button_id("applyBtn")

#wait for the apply button to be effective
time.sleep(7)


#scrolla fino alla fine della pagina, aspetta questo timeout per caricare i dati
SCROLL_PAUSE_TIME = 3

#scrollare main method
last_height = wd.execute_script("return document.body.scrollHeight")
while True:
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = wd.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

output_list = []

In [ ]:
type_tuple = ('Monthly','Quarterly','Semi-annual','Annual','Trailing Twelve Months')
#main scraper loop
tr_tuple = tuple(wd.find_elements_by_xpath('//*[@id="dividendsCalendarData"]/tbody//tr'))
for tr in progress_bar(tr_tuple):
  td_tuple = tr.find_elements_by_css_selector('td')
  if len(td_tuple) > 1:
    data = tuple([td_tuple[i].text for i in range(1,7)])
    output_list.append({'company': data[0], 
                        'Ex-dividend date' : data[1],
                        'Dividend' : data[2], 
                        'Type' : type_tuple[int(td_tuple[4].get_attribute('data-value'))-1],
                        'Payment date' : data[4],
                        'Yield' : data[5]})

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [ ]:
from pandas import DataFrame
df = DataFrame(output_list)
df.to_csv('sr_dividends_calendar_from_' + startdate.replace('/','') +  '_to_' + today_us_format.replace('/', '') + '.csv')

In [ ]:
df.head(15)

,company,Ex-dividend date,Dividend,Type,Payment date,Yield
0,AAR (AIR),"Jan 02, 2020",0.075,Quarterly,"Jan 14, 2020",1.96%
1,Ab Dynamics (ABDP),"Jan 02, 2020",2.79,Trailing Twelve Months,"Feb 13, 2020",0.16%
2,Aberdeen New Dawn (ABD),"Jan 02, 2020",4.3,Trailing Twelve Months,"Jan 31, 2020",1.76%
3,Aberdeen Smaller Companies High Inc (ASCIA),"Jan 02, 2020",8.25,Trailing Twelve Months,"Jan 24, 2020",4.45%
4,Adaro Energy ADR (ADOOY),"Jan 02, 2020",0.434299,Trailing Twelve Months,"Jan 30, 2020",13.57%
5,AllianceBernstein Global Hig Closed (AWF),"Jan 02, 2020",0.0655,Monthly,"Jan 17, 2020",6.34%
6,AllianceBernstein National Muniome (AFB),"Jan 02, 2020",0.0458083,Monthly,"Jan 17, 2020",3.91%
7,American Eagle Outfitters (AEO),"Jan 02, 2020",0.1375,Quarterly,"Jan 17, 2020",6.43%
8,American Express (AXP),"Jan 02, 2020",0.43,Quarterly,"Feb 10, 2020",1.59%
9,Anglo Pacific (APF),"Jan 02, 2020",9.63,Trailing Twelve Months,"Feb 13, 2020",5.58%
